In [31]:
import gymnasium as gym
import numpy as np

In [30]:
# env = gym.make("CartPole-v1")

# class CartPoleEnv(gym.Env):

#     metadata = {"render.modes": ["human", "rgb_array"], "video.frames_per_second": 50}

#     def __init__(self):

#         return 

import gym
env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset(seed=42)
print(f"{observation=}, {info=}")
for _ in range(1000):
#    action = policy(observation)  # User-defined policy function
   action = env.action_space.sample()
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()
env.close()

observation=array([ 0.0273956 , -0.00611216,  0.03585979,  0.0197368 ], dtype=float32), info={}


/Users/jadon/Documents/cart-pole/cart-pole-env/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 

In [58]:
'''
state space
0 |Cart Position | -4.8 4.8
1 |Cart Velocity |-Inf Inf 
2 |Pole Angle | ~ -0.418 rad (-24°) ~ 0.418 rad (24°) 
3 |Pole Angular Velocity |-Inf Inf
'''

state_space = env.observation_space.shape
action_space = env.action_space.shape

(4,)

In [61]:
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable
Qtable = initialize_q_table(state_space, action_space)


TypeError: 'tuple' object cannot be interpreted as an integer